In [1]:
import numpy as np

In [2]:
class LinearProgram(object):
    def __init__(self, c, A, b):
        self.c = np.asarray(c)
        self.A = np.asarray(A)
        self.b = np.asarray(b)
        self.m, self.n = self.A.shape
        assert self.c.shape == (self.n, 1), 'Please enter a vertical c vector with {} elements.'.format(self.n)
        assert self.b.shape == (self.m, 1), 'Please enter a vertical b vector with {} elements.'.format(self.m)
        assert np.all(self.b >= 0), 'Problem is not feasible at the origin.'
        self.non_slack = np.arange(self.n)
        self.slack = np.arange(self.m) + self.n
        self.index_list = np.concatenate((self.slack, self.non_slack))
        self.T = np.concatenate((np.concatenate((np.zeros((1, 1)), -self.c.T, np.zeros((1, self.m)), np.ones((1, 1))), 1), 
                                 np.concatenate((self.b, self.A, np.identity(self.m), np.zeros((self.m, 1))), 1)))
    def get_pivot_indices(self):
        pivot_j = None
        pivot_i = None
        ratio_list = []
        for j in range(self.n):
            if self.T[0, j + 1] < 0:
                pivot_j = j
                break
        for i in range(self.m):
            if self.T[i + 1, pivot_j + 1] > 0:
                ratio_list.append((i, self.T[i + 1, 0] / self.T[i + 1, pivot_j + 1]))
        assert ratio_list, 'The problem is unbounded and has no solutions.'
        pivot_i = min(ratio_list, key=lambda x: x[1])[0]
        return pivot_j, pivot_i
    
    def pivot(self):
        if np.all(self.T[0, 1:] >= 0):
            return False
        pivot_j, pivot_i = self.get_pivot_indices()
        index_i = None
        index_j = None
        for k, variable in enumerate(self.index_list):
            if variable == self.slack[pivot_i]:
                index_i = k
            elif variable == self.non_slack[pivot_j]:
                index_j = k
        self.index_list[index_i], self.index_list[index_j] = self.index_list[index_j], self.index_list[index_i]
        self.T[pivot_i + 1, :] /= self.T[pivot_i + 1, pivot_j + 1]
        for i in range(self.m + 1):
            if i == pivot_i + 1:
                pass
            else:
                self.T[i, :] -= self.T[i, pivot_j + 1] * self.T[pivot_i + 1, :]
        return True
    
    def solve(self):
        while self.pivot():
            pass
        basic_dict = dict(zip(self.index_list[:self.m], self.T[1:, 0]))
        non_basic_dict = dict(zip(self.index_list[self.m:], self.T[0, 1:self.n + 1]))
        max_val = self.T[0, 0]
        return max_val, basic_dict, non_basic_dict

In [3]:
lp1 = LinearProgram([[3], [2]], [[1, -1], [3, 1], [4, 3]], [[2], [5], [7]])
print(lp1.solve())

(5.2000000000000002, {2: 0.59999999999999964, 0: 1.6000000000000001, 1: 0.19999999999999982}, {3: 0.0, 4: 0.0})


In [4]:
data = np.load('../productMix.npz')
lp2 = LinearProgram(np.row_stack(data['p']), np.concatenate((data['A'], np.identity(4))), np.concatenate((np.row_stack(data['m']), np.row_stack(data['d']))))

In [5]:
solution = lp2.solve()
for i in range(4):
    print('Amount of good {} produced: {}'.format(i, solution[1][i]))

Amount of good 0 produced: 10.0
Amount of good 1 produced: 6.192982456140348
Amount of good 2 produced: 12.0
Amount of good 3 produced: 1.7894736842105292
